# Monte Carlo Estimator - $\pi$

## Example

Estimate $\pi$ using MC sampling. 

Consider a square $S$ of length $2r$, of which inside is a circle $C$ with radius $r$.

The area $A$ of each is:
$$
\begin{align*}
A_{S} &= 4r^2 \\
A_{C} &= \pi r^2 \\
\end{align*}
$$

Thus:
$$
\frac{\pi}{4} = \frac{A_C}{A_S} 
$$

So we can approximately expect that if we randomly sample (throw points) in $[0, r] \times [0, r]$, about $\frac{\pi}{4}$ of those land in the circle.

More precisely we calculate
$$
\pi = \int_{-r}^r \int_{-r}^r \mathbb{I}(x^2 + y^2 \leq r^2) \, dx \, dy
$$
and the MC estimator is
$$
\hat{\pi}_{N} \approx \frac{4}{N} \sum_{k=1}^N \mathbb{I}(x_k^2 + y_k^2 \leq r^2)
$$
where $\mathbb{I}$ is the indicator function.

In [1]:
import logging
import math
import random
from dataclasses import dataclass
from types import TracebackType
from typing import Optional, Type

from theoria.validor import TestCase, Validor

In [2]:
log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
if not log.hasHandlers():
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("[%(asctime)s] [%(levelname)s] %(message)s"))
    log.addHandler(handler)

In [3]:
@dataclass
class MonteCarloResult:
    result: float
    num_samples: int


class MonteCarloPi:
    def __init__(self, seed: int = 42):
        self.seed = seed
        self._original_state = None

    def __enter__(self):
        self._original_state = random.getstate()
        random.seed(self.seed)
        return self

    def __exit__(
        self,
        exc_type: Optional[Type[BaseException]],
        exc_value: Optional[BaseException],
        traceback: Optional[TracebackType],
    ) -> Optional[bool]:
        if self._original_state:
            random.setstate(self._original_state)
        return None

    def __call__(self, num_samples: int, radius: float = 1.0) -> float:
        inside_circle = 0
        for _ in range(num_samples):
            x = random.uniform(0, radius)  # noqa: S311
            y = random.uniform(0, radius)  # noqa: S311

            distance = x**2 + y**2
            if distance <= radius**2:
                inside_circle += 1

        return MonteCarloResult(
            result=4 * inside_circle / num_samples,
            num_samples=num_samples,
        )

    def __del__(self):
        if self._original_state:
            random.setstate(self._original_state)

# Tests

In [4]:
test_cases = [
    TestCase(
        input_data={"num_samples": int(1e2)},
        expected_output=math.pi,
        description="1e2 samples",
    ),
    TestCase(
        input_data={"num_samples": int(1e4)},
        expected_output=math.pi,
        description="1e4 samples",
    ),
    TestCase(
        input_data={"num_samples": int(1e6)},
        expected_output=math.pi,
        description="1e6 samples",
    ),
]

In [5]:
def comparison(actual: MonteCarloResult, expected: float) -> bool:
    # Slightly relaxed tolerance based on number of samples
    # Expect 1 / sqrt(N) convergence
    tolerance = 10 / math.sqrt(actual.num_samples)
    result = actual.result
    log.info(f"Comparing {result=} with {expected=} with relative {tolerance=}")
    return math.isclose(result, expected, rel_tol=tolerance)


with MonteCarloPi() as mc:
    Validor(mc).add_cases(test_cases).run(comparison=comparison)

[2025-12-01 18:00:29,059] [INFO] Comparing result=3.12 with expected=3.141592653589793 with relative tolerance=1.0
[2025-12-01 18:00:29,062] [INFO] Comparing result=3.1252 with expected=3.141592653589793 with relative tolerance=0.1
[2025-12-01 18:00:29,293] [INFO] Comparing result=3.140828 with expected=3.141592653589793 with relative tolerance=0.01
[2025-12-01 18:00:29,294] [INFO] All 3 tests passed for <__main__.MonteCarloPi object at 0x7c924a57f680>.
